In [89]:
# Imports
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import math
from VSM_IBVS import VSM_IBVS as V_IBVS
from math import pi
np.set_printoptions(linewidth=120, formatter={'float': lambda x: f"{0:8.4g}" if abs(x) < 1e-10 else f"{x:8.4g}"})


from machinevisiontoolbox.base import *
from machinevisiontoolbox import *
from spatialmath.base import *
from spatialmath import *

### Dataset generator class

In [90]:
class DatasetGenerator:
    def __init__(self):
        self.iterations = 10#1e4 # 10000 iterations
        self.steps = 80 # simulation steps
        self.inputs_hist = []
        self.outputs_hist = []
    
    def generate_dataset(self):
        for i in range(self.iterations):
            # Run the simulation
            ibvs = self.run_simulation()
            # Update dataset history
            self.update_dataset_hist(ibvs)
            # Print progress
            print(f"Progress: {i}/{self.iterations}")
            # Print shapes
            print(f"Inputs shape: {np.array(self.inputs_hist).shape}")
            print(f"Outputs shape: {np.array(self.outputs_hist).shape}")
            
        # Print final shapes
        self.store_dataset()
        
    def run_simulation(self):
        # Generate random pose
        pose = self.generate_random_pose()
        # Generate random desired pose
        desired_pose = self.generate_random_pose()
        # Initialize ibvs
        ibvs = self.build_ibvs(pose, desired_pose)
        try:
            ibvs.run(self.steps)
            return ibvs
        except:
            return self.run_simulation()
        
    def build_ibvs(self, pose, desired_pose):
        # Grid of points in the plane
        P = mkgrid(2, side=0.5, pose=SE3.Tz(3));

        # desired pose
        camera = CentralCamera.Default(pose=desired_pose);
        pd = 200 * np.array([[-1, -1, 1, 1], [-1, 1, 1, -1]]) + np.c_[camera.pp]

        # Defining the camera with default intrinsic parameters.
        camera = CentralCamera.Default(pose=pose);
        
        return IBVS(camera, P=P, p_d=pd)
    
    def generate_random_pose(self):
        pose = SE3.Rand()
        return pose
    
    def update_dataset_hist(self, ibvs):
        # For each point in the history store a new row in the inputs and outputs
        for i in range(len(ibvs.history)):
            # Initialize an empty list to store the flattened values
            hist = []
            hist.extend(ibvs.history[i].p.flatten()) # Flatten and extend the list with p values
            hist.extend(ibvs.p_star.flatten())       # Flatten and extend the list with p_star values
            
            # Append the row to the rows list
            self.inputs_hist.append(hist)
            self.outputs_hist.append(ibvs.history[i].vel.flatten())
            
    def store_dataset(self):
        np.save("dataset/inputs.npy", np.array(self.inputs_hist))
        np.save("dataset/outputs.npy", np.array(self.outputs_hist))

In [ ]:
dt_generator = DatasetGenerator()
dt_generator.generate_dataset()

In [93]:
np.array(dt_generator.inputs_hist).shape

(800, 16)